In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from vastai.api import VastClient
from io import StringIO
import time
from IPython.display import clear_output

In [3]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


## Search for Available Machines

In [15]:
def filter_offers():
    return [offer for offer in vast.search_offers('dph', 'dph<.12 inet_down>200', instance_type='on-demand') 
        if offer['gpu_name'] and offer['id'] not in []]

In [ ]:
for offer in filter_offers():
    print(offer['id'], offer['dph_total'], offer['gpu_name'], offer['inet_down'], offer['inet_down'])

## Start new Instance with Mongodb

In [ ]:
offer = 384633
# offer = filter_offers()[0]['id']
onstart = """#!/bin/sh
touch ~/.no_auto_tmux
echo onstart.sh Starting `date` >> /root/startup.log 2>&1
apt-get install -y mongodb-server git vim >> /root/startup.log 2>&1
service mongodb start >> /root/startup.log 2>&1
pip install --upgrade pip pipenv >> /root/startup.log 2>&1
echo onstart.sh Completed at `date` >> /root/startup.log
"""
resp=vast.create_instance(offer, disk=20, onstart_cmd=onstart, python_utf8=True, lang_utf8=True,
                          image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
mongo_instance_id = resp['new_contract']

mongo_instance = vast.get_instance(mongo_instance_id, retries=4).wait_until_running()
remote=mongo_instance.pb_remote

print(mongo_instance.pb_remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

In [ ]:
print(mongo_instance.pb_remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

In [ ]:
print(mongo_instance.ssh_connection_command)

In [ ]:
# onstart = "#!/bin/sh\n"\
#           "touch ~/.no_auto_tmux\n"\
#           "apt-get install -y mongodb-server\n"\
#           "mkdir -p ~/.ssh\n"\
#           "ssh-keygen  -P '' -f ~/.ssh/mongo\n"
# resp=vast.create_instance(filter_offers()[0]['id'], disk=1, onstart_cmd=onstart,
#                           image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
# worker_instance_id = resp['new_contract']

# time.sleep(30)
# worker_instance = vast.get_instance(worker_instance_id)

## Install model and download data

In [5]:
instance=vast.get_instance(400808)
remote=instance.pb_remote
print(instance.ssh_connection_command)

ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai


In [ ]:
install_script_target='/root/install_script.sh'
install_script = StringIO("""#!/bin/sh
INSTALL_LOG=/root/install.log
echo avian_vocalizations INSTALL BEGIN `date` >> $INSTALL_LOG 2>&1
cd ~
[ ! -d avian_vocalizations ] && git clone https://github.com/samhiatt/avian_vocalizations.git  >> $INSTALL_LOG 2>&1
cd avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
pipenv install >> $INSTALL_LOG 2>&1
pipenv run python setup.py develop >> $INSTALL_LOG 2>&1
pipenv run download_data >> $INSTALL_LOG 2>&1
echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
""")
# monitor_output(mongo_instance, install_script)
remote.sftp.putfo(install_script,install_script_target)
remote['chmod']('+x',install_script_target)
p=remote.session().popen(install_script_target)
p

In [8]:
while True:
    clear_output(wait=True)
    print(remote['tail']('/root/install.log','-n5'))
    try:
        ret = p.proc.poll()
    except NameError:
        break
    if ret is not None:
        print("Install process has exited with code:",p.proc.poll())
        break
    time.sleep(1)

Installing download_data script to /root/.local/share/virtualenvs/avian_vocalizations--GzUH1gw/bin

Installed /root/avian_vocalizations
Processing dependencies for avian-vocalizations==0.0.1
Finished processing dependencies for avian-vocalizations==0.0.1



In [9]:
hyperopt_worker_log="/root/hyperopt_worker.log"
start_hyperopt_worker_target='/root/avian_vocalizations/start_hyperopt_worker.sh'
start_hyperopt_worker="#!/bin/sh\ncd /root/avian_vocalizations\n"\
    "export LC_ALL=C.UTF-8\n"\
    "export LANG=C.UTF-8\n"\
    "nohup pipenv run hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations "\
    " --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=."\
    " >> %s 2>&1\n"%hyperopt_worker_log

print(start_hyperopt_worker)
remote.sftp.putfo(StringIO(start_hyperopt_worker),start_hyperopt_worker_target)
remote['chmod']('+x',start_hyperopt_worker_target)
pworker=remote.session().popen(start_hyperopt_worker_target)

#!/bin/sh
cd /root/avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
nohup pipenv run hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations  --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=. >> /root/hyperopt_worker.log 2>&1



In [10]:
while True:
    clear_output(wait=True)
    print(remote['tail'](hyperopt_worker_log,'-n5'))
    ret = pworker.proc.poll()
    if ret is not None:
        break
    time.sleep(1)
print("Hyperopt worker process ended with code",ret)

  File "/root/.local/share/virtualenvs/avian_vocalizations--GzUH1gw/lib/python3.6/site-packages/sklearn/utils/validation.py", line 56, in _assert_all_finite
    raise ValueError(msg_err.format(type_err, X.dtype))
ValueError: Input contains infinity or a value too large for dtype('float32').
INFO:hyperopt.mongoexp:exiting with N=9223372036854775806 after 1 consecutive exceptions
INFO:hyperopt.mongoexp:no job found, sleeping for 3.5s

Hyperopt worker process ended with code 0


In [ ]:
# vast.stop_all_instances()